In [25]:
import cv2
import numpy as np
import imutils
import os
from commonfunctions import *
from sklearn.svm import OneClassSVM
from skimage.feature import hog
import joblib

In [55]:
def train_svm():
    features = []
    directory = "Results4"
    for file in os.listdir(directory):
        img = cv2.imread(f"{directory}\\{file}")
        # label_file = open(f"Dataset\\Vehicles Labeling\\{i:04d}.txt", "r")
        # label_data = label_file.read()
        # label_file.close()
        # label_data = label_data.strip()
        # label_data = label_data.replace('\n', '')
        # label_data = label_data.split(" ")
        # label_data = list(map(float, label_data))
        # if img is None:
        #     continue
        # h, w, _ = img.shape
        # x_center, y_center, box_width, box_height = label_data[1], label_data[2], label_data[3], label_data[4]
        # # Convert YOLO format to corner coordinates
        # x1 = int((x_center - box_width / 2) * w)
        # y1 = int((y_center - box_height / 2) * h)
        # x2 = int((x_center + box_width / 2) * w)
        # y2 = int((y_center + box_height / 2) * h)
        # plate = img[y1:y2, x1:x2]
        plate = img
        plate = cv2.resize(plate, (128, 64))
        plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
        hog_features = hog(plate, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', transform_sqrt=True, feature_vector=True)    
        features.append(hog_features)
        
    features_train = np.array(features)
    clf = OneClassSVM(kernel='rbf', gamma='scale', nu=0.1)
    clf.fit(features_train)
    
    # Save the model
    joblib.dump(clf, 'svm.pkl')
    return clf
        

In [56]:
# clf = train_svm()

In [57]:
model = joblib.load('svm.pkl')

In [59]:
def test_svm(num_samples,model):
    max_samples = 2000

    random_numbers = np.random.randint(1, max_samples, num_samples)
    features = []
    for i in random_numbers:
        img = cv2.imread(f"Dataset\\Vehicles\\{i:04d}.jpg")
        label_file = open(f"Dataset\\Vehicles Labeling\\{i:04d}.txt", "r")
        label_data = label_file.read()
        label_file.close()
        label_data = label_data.strip()
        label_data = label_data.replace('\n', '')
        label_data = label_data.split(" ")
        label_data = list(map(float, label_data))
        
        h, w, _ = img.shape
        x_center, y_center, box_width, box_height = label_data[1], label_data[2], label_data[3], label_data[4]
        # Convert YOLO format to corner coordinates
        x1 = int((x_center - box_width / 2) * w)
        y1 = int((y_center - box_height / 2) * h)
        x2 = int((x_center + box_width / 2) * w)
        y2 = int((y_center + box_height / 2) * h)
        plate = img[y1:y2, x1:x2]
        plate = cv2.resize(plate, (128, 64))
        plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
        hog_features = hog(plate, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', transform_sqrt=True, feature_vector=True)    
        hog_features = hog_features.reshape(1, -1)
        is_plate = model.predict(hog_features)
        
        print("Plate" if is_plate == 1 else "Not a Plate")

In [60]:
test_svm(10,model)

Plate
Not a Plate
Plate
Not a Plate
Plate
Plate
Not a Plate
Not a Plate
Plate
Plate
